### Compute pRF threshold maps
- PRF Amplitude: positive only
- PRF R2: between 0 and 1
- PRF Eccentricity: between 0.05 and 15 dva
- PRF SD (size): between 0.05 and 15 dva 

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import nibabel as nb
import warnings
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'group']
tasks = ['FullScreen', 'FullScreenAttendFix', 'FullScreenAttendBar']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

# Threshold values
amp_th = 0
r2_th = [0, 1]
ecc_th = [0.05, 15]
sd_th = [0.05, 15]

In [2]:
# Create pRF threshold masks
for task in tasks:
    for subject in subjects:
    
        # define folders
        fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
        mask_dir = '{}/{}/masks'.format(pp_dir, subject)
        
        # Load fit parameters
        r2_im = nb.load('{}/{}_task-{}_par-r2.nii.gz'.format(fit_dir,subject,task))
        ecc_im = nb.load('{}/{}_task-{}_par-ecc.nii.gz'.format(fit_dir,subject,task))
        sd_im = nb.load('{}/{}_task-{}_par-sd.nii.gz'.format(fit_dir,subject,task))
        amp_im = nb.load('{}/{}_task-{}_par-amplitude.nii.gz'.format(fit_dir,subject,task))

        # Create param matrix
        r2_idx, ecc_idx, sd_idx, amp_idx, mask_idx = 0, 1, 2, 3, 4
        param_mat = np.zeros((r2_im.shape[0],r2_im.shape[1],r2_im.shape[2],5))*np.nan
        param_mat[...,r2_idx] = r2_im.get_fdata()
        param_mat[...,ecc_idx] = ecc_im.get_fdata()
        param_mat[...,sd_idx] = sd_im.get_fdata()
        param_mat[...,amp_idx] = amp_im.get_fdata()
 
        # Threshold param matrix by masking R2
        param_mat_th = param_mat
        amp_down =  param_mat_th[...,amp_idx] > amp_th
        r2_th_down, r2_th_up = param_mat_th[...,r2_idx] >= r2_th[0],  param_mat_th[...,r2_idx] <= r2_th[1]
        ecc_th_down, ecc_th_up = param_mat_th[...,ecc_idx] >= ecc_th[0], param_mat_th[...,ecc_idx] <= ecc_th[1]
        sd_th_down, sd_th_up = param_mat_th[...,sd_idx] >= sd_th[0], param_mat_th[...,sd_idx] <= sd_th[1]
        all_th = np.array((amp_down, r2_th_down, r2_th_up, ecc_th_down, ecc_th_up, sd_th_down, sd_th_up)) 
        
        # create threhsold mask
        param_mat[np.logical_and.reduce(all_th)==False, mask_idx] = np.nan
        param_mat[np.logical_and.reduce(all_th)==True, mask_idx] = 1
        
        # save threshold masks
        th_mask_file = "{}/{}_task-{}_prf_threshold.nii.gz".format(mask_dir, subject, task)
        print('saving {}'.format(th_mask_file))
        th_mask_img = nb.Nifti1Image(dataobj=param_mat[...,mask_idx], affine=r2_im.affine, header=r2_im.header)
        th_mask_img.to_filename(th_mask_file)

saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-001/masks/sub-001_task-FullScreen_prf_threshold.nii.gz
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-002/masks/sub-002_task-FullScreen_prf_threshold.nii.gz
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-003/masks/sub-003_task-FullScreen_prf_threshold.nii.gz
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-004/masks/sub-004_task-FullScreen_prf_threshold.nii.gz
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-005/masks/sub-005_task-FullScreen_prf_threshold.nii.gz
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-006/masks/sub-006_task-FullScreen_prf_threshold.nii.gz
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-007/masks/sub-007_task-FullScreen_prf_threshold.nii.gz
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-008/masks/sub-008_task-FullScreen_prf_t